# Digital Phenotyping and Mental Health Risk Prediction Using Wearable Device Data

### --- Introduction ---
### This notebook follows a methodological approach to explore, preprocess, and model data collected from wearable devices and mental health questionnaires. The goal is to predict mental health risk using physiological data and advanced machine learning techniques.

**Table of Contents:**
1. [Setup and Library Installation](#setup)
2. [Data Loading and Exploratory Data Analysis (EDA)](#eda)
3. [Data Preprocessing](#preprocessing)
4. [Data Transformation](#transformation)
5. [Machine Learning Model Development](#modeling)
6. [Model Evaluation and Visualization](#evaluation)
7. [Model Interpretability](#interpretability)
8. [Alternative Modeling with Dimensionality Reduction](#dimensionality)
9. [Comparison of Models with Different PCA Components](#comparison)
10. [Data Integrity and Minimization](#integrity)

<a name="setup"></a>
## 1. Setup and Library Installation

In [ ]:
# Mount Google Drive to access data files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# Install necessary libraries quietly to avoid cluttering the output
!pip install pyts seaborn dask tf-keras-vis shap lime fancyimpute --quiet


In [ ]:
# Import necessary libraries for data manipulation and visualization
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import timedelta
import dask.dataframe as dd

# Import libraries for data preprocessing and modeling
from sklearn.impute import SimpleImputer
from scipy import stats
from sklearn.utils import shuffle
from pyts.image import GramianAngularField
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Import libraries for building the neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Import libraries for interpretability
import shap
import lime
from lime import lime_image
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore

# Import MICE imputation method from scikit-learn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plot styles for better aesthetics
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:
# Base path to data files in Google Drive
base_path = "/content/drive/MyDrive/stress_master/"


In [ ]:
# Questionnaire files and their respective timestamp columns
questionnaire_files_info = {
    "patient_health_questionnaire_phq9.csv": "phq9_ts",
    "generalized_anxiety_disorder_scale_gad7.csv": "gad_ts",
    "perceived_stress_scale_pss4.csv": "pss4_ts"
}

# Digital Health Technology (DHT) files (Physiological Data from Wearable Devices)
dht_files = [
    "garmin_epoch_run.csv",
    "oura_extension_readiness.csv",
    "oura_readiness.csv",
    "garmin_epoch_walk.csv",
    "oura_extension_sleep.csv",
    "garmin_epoch_idle.csv",
    "oura_sleep.csv",
    "oura_extension_activity.csv"
]

# Valid participant IDs (ensures consistency across datasets)
valid_ids = {
    'U-B77UDKRYXV85ESYFKNQ5', 'U-5U51AGTFQ3ZQP6LHASV8', 'U-BKQPVE7SQ6S4E3F2TCAT',
    'U-GS4NJ3G9ZVUETDF1847D', 'U-V1TJFFPQALNM9G3LRTSP', 'U-ZW5VUY46VKDERT8XM6Z4',
    'U-HMS81C96UB7MPJJ4CSTH', 'U-8RBGSLAPG8YLY6FHJW75', 'U-J692A9AKCT7LM13RVH1K',
    'U-QWKQ632DVK5WBMPQTL3M', 'U-D6M4TDMFSD1WQL88LRN8', 'U-W41B6L41VS1LGYMNPRCU',
    'U-PZ5PSRW2SVQHNDNNLM7W', 'U-S1JFKPZWUG7K2L6DMBQU', 'U-X8Q3A3Z18ZBAXJ6DMZ5W',
    'U-8ZKFQ6KLTMU17TT6DBF1', 'U-SWLQR3AXMCT7KURBMV8H', 'U-YY1J4CAB6VNDKUFHJ99E',
    'U-3N1NZWHWN7Y96Z3Z1A9J', 'U-8CM21J68ZTPZHRH5REXV', 'U-TA9DZH13P9C7LWU9DRTN',
    'U-FY2M2PN4EFRQYN9R5CXY', 'U-QZLLZBKC13LNFT9NZASU', 'U-L57KAZZ2HNALN6UZW1PV',
    'U-YT5AF8GSQA3WKACCU6H8', 'U-T4GSS7ESBLQWRCYT4RDX', 'U-6N9FECGL19X1E4LXBSM3',
    'U-XZ33PFUXKV7WPYR3XGC7', 'U-NZKTR5L6XXPT4BNQF3NQ', 'U-7KJCJE27RESNM2M74TF4',
    'U-ZUY395L4QK8TQXMLTJNH', 'U-1M1PWFRM99UKVZBE4JN8', 'U-51YQ41XX3C54B86EM775',
    'U-SSQ22TL31ZVQ5FT87FYA', 'U-D9UX49JDSB2RK49X12HS', 'U-FZFXQL6T5J3K7QHZJRTA',
    'U-J8E38GJFGUJBMY7HFQ4V', 'U-ACCEE9K6JQL6XBHFRZHE', 'U-XZ7U958ZLVPHTUVFSZW7',
    'U-X4XN7213NM6V52ERNC3L', 'U-YY1AMRSCJ1N8FTJE8YH6', 'U-XLBUMB67W4MZCG847GZ9',
    'U-7ZYS5MWSE339X5Q4PFPM', 'U-YQEHZM1ZP3XCSHVBP11V', 'U-DBY492Q47FNB3C68CHC1',
    'U-GFN8A66UFG8K3UFX3XQK', 'U-B9EERJ5XGKFNXMUP76P6', 'U-EAJF8BV1CZV7TU8Q4A97',
    'U-BL3W3BNN4RNC4Z29DZRS', 'U-V9DDVJLUL1N7BSYMLW76', 'U-JQPCQ457SWNQKQPHBARR',
    'U-7N4QAAKGRRXGV7V66NSG'
}

# Define feature sets for HRV and sleep metrics
hrv_columns = ['score_hrv_balance', 'rmssd', 'hr_lowest']
sleep_columns = ['score_sleep_balance', 'score_total', 'deep', 'light', 'rem', 'efficiency', 'onset_latency', 'hr_average']
outcome_columns = ['phq9_total', 'gad_total', 'pss4_total']


<a name="eda"></a>
## 2. Data Loading and Exploratory Data Analysis (EDA)

In this section, we will load the questionnaire and DHT data, and perform initial exploratory data analysis to understand the datasets.

In [ ]:
# Function to load questionnaire data efficiently
def load_questionnaire_files(base_path, questionnaire_files_info, chunksize=500000):
    questionnaire_dfs = []
    for file, timestamp_col in questionnaire_files_info.items():
        file_path = os.path.join(base_path, file)
        if not os.path.exists(file_path):
            continue  # Skip if the file doesn't exist
        try:
            # Read data in chunks to manage memory usage
            chunk_iter = pd.read_csv(
                file_path,
                chunksize=chunksize,
                dtype={'participant_id': 'object'},
                parse_dates=[timestamp_col]
            )
            for chunk in chunk_iter:
                questionnaire_dfs.append(chunk)
        except Exception as e:
            print(f"Error loading {file}: {e}")
    if not questionnaire_dfs:
        return pd.DataFrame()
    merged_questionnaires = pd.concat(questionnaire_dfs, ignore_index=True)
    # Compute total scores for questionnaires if not already present
    if 'phq9_total' not in merged_questionnaires.columns and any(col.startswith('phq9_') for col in merged_questionnaires.columns):
        phq9_cols = [col for col in merged_questionnaires.columns if col.startswith('phq9_') and col != 'phq9_ts']
        merged_questionnaires['phq9_total'] = merged_questionnaires[phq9_cols].sum(axis=1)
    if 'gad_total' not in merged_questionnaires.columns and any(col.startswith('gad_') for col in merged_questionnaires.columns):
        gad_cols = [col for col in merged_questionnaires.columns if col.startswith('gad_') and col != 'gad_ts']
        merged_questionnaires['gad_total'] = merged_questionnaires[gad_cols].sum(axis=1)
    if 'pss4_total' not in merged_questionnaires.columns and any(col.startswith('pss4_') for col in merged_questionnaires.columns):
        pss4_cols = [col for col in merged_questionnaires.columns if col.startswith('pss4_') and col != 'pss4_ts']
        merged_questionnaires['pss4_total'] = merged_questionnaires[pss4_cols].sum(axis=1)
    return merged_questionnaires


In [ ]:
# Function to load DHT data efficiently
def load_dht_files(base_path, dht_files, chunksize=500000):
    dht_dfs = []
    for file in dht_files:
        file_path = os.path.join(base_path, file)
        if not os.path.exists(file_path):
            continue  # Skip if the file doesn't exist
        try:
            # Read data in chunks to manage memory usage
            chunk_iter = pd.read_csv(
                file_path,
                chunksize=chunksize,
                dtype={'participant_id': 'object'},
                parse_dates=['summary_date']
            )
            for chunk in chunk_iter:
                dht_dfs.append(chunk)
        except Exception as e:
            print(f"Error loading {file}: {e}")
    if not dht_dfs:
        return pd.DataFrame()
    merged_dht = pd.concat(dht_dfs, ignore_index=True)
    return merged_dht


In [ ]:
# Load questionnaire data
questionnaire_data = load_questionnaire_files(base_path, questionnaire_files_info)


In [ ]:
# Load DHT data
dht_data = load_dht_files(base_path, dht_files)


<a name="preprocessing"></a>
## 3. Data Preprocessing

We will preprocess the data by merging datasets, handling missing values using MICE imputation, standardizing timestamps, and creating risk labels based on questionnaire scores.

In [ ]:
# Merge and clean data only if both datasets are available
if not questionnaire_data.empty and not dht_data.empty:
    # Select relevant columns from questionnaire data
    questionnaire_columns = ['participant_id', 'phq9_ts', 'gad_ts', 'pss4_ts', 'phq9_total', 'gad_total', 'pss4_total']
    available_questionnaire_columns = [col for col in questionnaire_columns if col in questionnaire_data.columns]
    questionnaire_data = questionnaire_data[available_questionnaire_columns]

    # Select relevant columns from DHT data
    selected_dht_columns = ['participant_id', 'summary_date'] + hrv_columns + sleep_columns
    available_dht_columns = [col for col in selected_dht_columns if col in dht_data.columns]
    dht_data = dht_data[available_dht_columns]

    # Merge datasets on 'participant_id'
    merged_data = pd.merge(
        dht_data,
        questionnaire_data,
        on='participant_id',
        how='inner'  # Ensures only participants present in both datasets are included
    )
    # Filter for valid participant IDs
    merged_data = merged_data[merged_data['participant_id'].isin(valid_ids)]
    # Drop rows where all critical timestamps are missing
    critical_columns = ['summary_date', 'phq9_ts', 'gad_ts', 'pss4_ts']
    merged_data.dropna(subset=critical_columns, how='all', inplace=True)
else:
    merged_data = pd.DataFrame()


In [ ]:
# Merge datasets
merged_data = pd.merge(
    dht_data,
    questionnaire_data,
    on='participant_id',
    how='inner'
)

# Strip whitespace from column names
merged_data.columns = merged_data.columns.str.strip()

# Rename 'summary_date' to 'timestamp_dht'
if 'summary_date' in merged_data.columns:
    merged_data.rename(columns={'summary_date': 'timestamp_dht'}, inplace=True)
    print("'summary_date' column successfully renamed to 'timestamp_dht'.")
else:
    print("'summary_date' column not found after stripping whitespace.")

# Verify columns after renaming
print("Columns after renaming:", merged_data.columns.tolist())

# Process timestamps
if 'timestamp_dht' in merged_data.columns:
    timestamp_cols = ['timestamp_dht', 'phq9_ts', 'gad_ts', 'pss4_ts']
    for col in timestamp_cols:
        if col in merged_data.columns:
            merged_data[col] = pd.to_datetime(merged_data[col], utc=True, errors='coerce')
            merged_data[col] = merged_data[col].dt.floor('D')  # Retain only the date part
    # Drop rows with missing 'timestamp_dht'
    merged_data.dropna(subset=['timestamp_dht'], inplace=True)
    print("Merged data shape after timestamp processing:", merged_data.shape)
else:
    print("'timestamp_dht' column is missing after renaming. Cannot proceed.")

# Verify columns after timestamp processing
print("Columns after timestamp processing:", merged_data.columns.tolist())

# Filter for valid participant IDs
print("Merged data shape before filtering valid IDs:", merged_data.shape)
merged_data = merged_data[merged_data['participant_id'].isin(valid_ids)]
print("Merged data shape after filtering valid IDs:", merged_data.shape)

# Verify columns after filtering
print("Columns after filtering valid IDs:", merged_data.columns.tolist())

# Check if 'timestamp_dht' is still present
if 'timestamp_dht' in merged_data.columns:
    print("'timestamp_dht' column is still present after filtering.")
else:
    print("'timestamp_dht' column is missing after filtering.")


'summary_date' column successfully renamed to 'timestamp_dht'.
Columns after renaming: ['participant_id', 'timestamp_dht', 'score_hrv_balance', 'rmssd', 'hr_lowest', 'score_sleep_balance', 'score_total', 'deep', 'light', 'rem', 'efficiency', 'onset_latency', 'hr_average', 'phq9_ts', 'gad_ts', 'pss4_ts', 'phq9_total', 'gad_total', 'pss4_total']
Merged data shape after timestamp processing: (11589006, 19)
Columns after timestamp processing: ['participant_id', 'timestamp_dht', 'score_hrv_balance', 'rmssd', 'hr_lowest', 'score_sleep_balance', 'score_total', 'deep', 'light', 'rem', 'efficiency', 'onset_latency', 'hr_average', 'phq9_ts', 'gad_ts', 'pss4_ts', 'phq9_total', 'gad_total', 'pss4_total']
Merged data shape before filtering valid IDs: (11589006, 19)
Merged data shape after filtering valid IDs: (5965093, 19)
Columns after filtering valid IDs: ['participant_id', 'timestamp_dht', 'score_hrv_balance', 'rmssd', 'hr_lowest', 'score_sleep_balance', 'score_total', 'deep', 'light', 'rem', 'e

In [ ]:
# Ensure 'summary_date' is correctly renamed to 'timestamp_dht'
if 'summary_date' in merged_data.columns:
    merged_data.rename(columns={'summary_date': 'timestamp_dht'}, inplace=True)

# Convert timestamps and check for missing values
timestamp_cols = ['timestamp_dht', 'phq9_ts', 'gad_ts', 'pss4_ts']
for col in timestamp_cols:
    if col in merged_data.columns:
        merged_data[col] = pd.to_datetime(merged_data[col], utc=True, errors='coerce')
        print(f"Missing values in {col} after conversion: {merged_data[col].isnull().sum()}")

# Check if all 'timestamp_dht' values are missing
if merged_data['timestamp_dht'].isnull().all():
    print("All 'timestamp_dht' values are missing after conversion.")
    # Investigate why and address accordingly
else:
    # Drop rows with missing 'timestamp_dht' if necessary
    merged_data.dropna(subset=['timestamp_dht'], inplace=True)
    print("Shape after dropping missing 'timestamp_dht':", merged_data.shape)


Missing values in timestamp_dht after conversion: 0
Missing values in phq9_ts after conversion: 11103618
Missing values in gad_ts after conversion: 11122210
Missing values in pss4_ts after conversion: 10672671
Shape after dropping missing 'timestamp_dht': (11589006, 19)


In [ ]:
# Proceed only if merged_data is not empty
if not merged_data.empty:
    # Convert 'timestamp_dht' to datetime
    merged_data['timestamp_dht'] = pd.to_datetime(merged_data['timestamp_dht'], utc=True, errors='coerce')

    # Retain only date information
    merged_data['timestamp_dht'] = merged_data['timestamp_dht'].dt.floor('D')

    # Drop rows with missing 'timestamp_dht' (none in your case)
    merged_data.dropna(subset=['timestamp_dht'], inplace=True)

    # Print the shape after processing 'timestamp_dht'
    print("Shape after processing 'timestamp_dht':", merged_data.shape)

    # Clean numeric columns
    numeric_cols = hrv_columns + sleep_columns + outcome_columns
    available_numeric_cols = [col for col in numeric_cols if col in merged_data.columns]

    # Convert columns to numeric type
    merged_data[available_numeric_cols] = merged_data[available_numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Check for missing values in numeric columns
    total_missing = merged_data[available_numeric_cols].isnull().sum().sum()
    print(f"Total missing values in numeric columns before imputation: {total_missing}")

    # Proceed with MICE imputation
    print("Performing MICE imputation on missing values...")
    mice_imputer = IterativeImputer(max_iter=10, random_state=42)
    merged_data[available_numeric_cols] = mice_imputer.fit_transform(merged_data[available_numeric_cols])

    # Verify that missing values have been imputed
    total_missing_after = merged_data[available_numeric_cols].isnull().sum().sum()
    print(f"Total missing values in numeric columns after imputation: {total_missing_after}")
else:
    print("Merged data is empty after merging. Cannot proceed.")


Shape after processing 'timestamp_dht': (11589006, 19)
Total missing values in numeric columns before imputation: 117082628
Performing MICE imputation on missing values...
Total missing values in numeric columns after imputation: 0


In [ ]:
# 1. Check DataFrames are not empty
print("dht_data shape:", dht_data.shape)
print("questionnaire_data shape:", questionnaire_data.shape)

# 2. Ensure 'participant_id' columns exist
if 'participant_id' not in dht_data.columns:
    print("Error: 'participant_id' column missing in dht_data")
if 'participant_id' not in questionnaire_data.columns:
    print("Error: 'participant_id' column missing in questionnaire_data")

# 3. Normalize 'participant_id's
dht_data['participant_id'] = dht_data['participant_id'].astype(str).str.strip().str.upper()
questionnaire_data['participant_id'] = questionnaire_data['participant_id'].astype(str).str.strip().str.upper()

# 4. Check for common 'participant_id's
common_ids = set(dht_data['participant_id']).intersection(set(questionnaire_data['participant_id']))
print("Number of common participant IDs after normalization:", len(common_ids))

# 5. Proceed with merging if common IDs exist
if len(common_ids) > 0:
    merged_data = pd.merge(
        dht_data,
        questionnaire_data,
        on='participant_id',
        how='inner'
    )
    print("Merged data shape after merging:", merged_data.shape)
else:
    print("No common participant IDs found. Cannot proceed with merging.")


dht_data shape: (422127, 13)
questionnaire_data shape: (7622, 7)
Number of common participant IDs after normalization: 341
Merged data shape after merging: (11589006, 19)


In [ ]:
    # Define risk thresholds based on clinical guidelines
    phq9_threshold = 10  # Moderate to severe depression
    gad_threshold = 10   # Moderate to severe anxiety
    pss4_threshold = 7   # High perceived stress

    # Create binary risk labels
    if 'phq9_total' in merged_data.columns:
        merged_data['risk_label_phq9'] = np.where(merged_data['phq9_total'] >= phq9_threshold, 'Risk', 'No Risk')
    if 'gad_total' in merged_data.columns:
        merged_data['risk_label_gad'] = np.where(merged_data['gad_total'] >= gad_threshold, 'Risk', 'No Risk')
    if 'pss4_total' in merged_data.columns:
        merged_data['risk_label_pss'] = np.where(merged_data['pss4_total'] >= pss4_threshold, 'Risk', 'No Risk')

    # Combine risk labels into a single risk category
    merged_data['risk_category'] = merged_data[['risk_label_phq9', 'risk_label_gad', 'risk_label_pss']].apply(
        lambda x: 'Risk' if any(label == 'Risk' for label in x) else 'No Risk', axis=1)


In [ ]:
    # Visualize the distribution of risk categories
    plt.figure(figsize=(10, 6))
    sns.countplot(x='risk_category', data=merged_data)
    plt.title('Distribution of Risk Categories')
    plt.xlabel('Risk Category')
    plt.ylabel('Count')
    plt.show()


In [ ]:
    # Filter data for the study period (May 2nd to December 1st, 2020)
    merged_data = merged_data[(merged_data['timestamp_dht'] >= '2020-05-02') &
                              (merged_data['timestamp_dht'] <= '2020-12-01')]
    # Sort data by timestamp for time-series analysis
    merged_data = merged_data.sort_values(by='timestamp_dht')


<a name="transformation"></a>
## 4. Data Transformation

We will apply dynamic windowing and transform time-series data into images using Gramian Angular Field (GAF) for input into a Convolutional Neural Network (CNN).

In [ ]:
# Function to process each participant's data and create GAF images
def process_participant(participant_data, metric, window_size=7):
    X_gaf = []
    y_gaf = []
    # Iterate over the data using a sliding window approach
    for start in range(0, len(participant_data) - window_size + 1, 1):
        end = start + window_size
        window_data = participant_data.iloc[start:end]
        window_metric = window_data[metric].values
        if np.any(np.isnan(window_metric)):
            continue  # Skip if any values are missing in the window
        # Apply Gramian Angular Field transformation
        gaf = GramianAngularField(image_size=min(window_size, 32), method='summation')
        gaf_image = gaf.fit_transform(window_metric.reshape(1, -1))
        # Resize the image to 32x32 pixels
        resized_gaf_image = resize(gaf_image[0], (32, 32), anti_aliasing=True)
        X_gaf.append(resized_gaf_image)
        # Determine the majority risk label in the window
        window_risk_labels = window_data['risk_category']
        majority_label = window_risk_labels.mode()[0]
        y_gaf.append(1 if majority_label == 'Risk' else 0)
    return X_gaf, y_gaf


In [ ]:
# Function to create GAF images for all participants using sliding windows
from concurrent.futures import ThreadPoolExecutor

def create_sliding_window_gaf(data, metric, window_size=7, batch_size=20000):
    participants = data['participant_id'].unique()
    X_gaf = []
    y_gaf = []
    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_participant, data[data['participant_id'] == participant], metric, window_size) for participant in participants]
        for future in futures:
            X_batch, y_batch = future.result()
            X_gaf.extend(X_batch)
            y_gaf.extend(y_batch)
            # Yield batches to manage memory usage
            if len(X_gaf) >= batch_size:
                yield np.array(X_gaf), np.array(y_gaf)
                X_gaf, y_gaf = [], []
    # Yield any remaining data
    if X_gaf:
        yield np.array(X_gaf), np.array(y_gaf)


In [ ]:
# Generate GAF images for the 'score_hrv_balance' metric
X_gaf_hrv_balance = []
y_gaf_hrv_balance = []
for X_batch, y_batch in create_sliding_window_gaf(merged_data, metric='score_hrv_balance'):
    X_gaf_hrv_balance.append(X_batch)
    y_gaf_hrv_balance.append(y_batch)
# Concatenate all batches into a single array
X_gaf_hrv_balance = np.concatenate(X_gaf_hrv_balance, axis=0)
y_gaf_hrv_balance = np.concatenate(y_gaf_hrv_balance, axis=0)


<a name="modeling"></a>
## 5. Machine Learning Model Development

We will develop a CNN model to classify the risk of mental health issues based on the transformed GAF images.

In [ ]:
# Shuffle the dataset to ensure randomness
X_gaf_hrv_balance, y_gaf_hrv_balance = shuffle(X_gaf_hrv_balance, y_gaf_hrv_balance, random_state=42)


In [ ]:
# Split data into training and testing sets (80% training, 20% testing)
X_train_hrv_balance, X_test_hrv_balance, y_train_hrv_balance, y_test_hrv_balance = train_test_split(
    X_gaf_hrv_balance, y_gaf_hrv_balance, test_size=0.2, random_state=42, stratify=y_gaf_hrv_balance
)

In [ ]:
# Reshape data for input into the CNN (add channel dimension)
X_train_hrv_balance = X_train_hrv_balance.reshape(-1, 32, 32, 1).astype('float32')
X_test_hrv_balance = X_test_hrv_balance.reshape(-1, 32, 32, 1).astype('float32')


In [ ]:
# Define the Convolutional Neural Network (CNN) architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),  # First convolutional layer
    MaxPooling2D(2, 2),  # First pooling layer
    Conv2D(64, (3, 3), activation='relu'),  # Second convolutional layer
    MaxPooling2D(2, 2),  # Second pooling layer
    Flatten(),  # Flatten the feature maps into a 1D vector
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Dropout layer to prevent overfitting
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])


In [ ]:
# Compile the model with optimizer, loss function, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Define callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


In [ ]:
# Train the model with validation data and callbacks
history = model.fit(X_train_hrv_balance, y_train_hrv_balance,
                    epochs=30,
                    validation_data=(X_test_hrv_balance, y_test_hrv_balance),
                    batch_size=64,
                    callbacks=[early_stopping, reduce_lr])


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_hrv_balance, y_test_hrv_balance)
print(f'HRV Balance Model - Test Accuracy: {test_accuracy}, Loss: {test_loss}')


<a name="evaluation"></a>
## 6. Model Evaluation and Visualization

We will evaluate the model using metrics like confusion matrix, classification report, ROC curve, and visualize the training history.

In [ ]:
# Generate predictions on the test set
y_pred_hrv_balance_probs = model.predict(X_test_hrv_balance)
y_pred_hrv_balance = (y_pred_hrv_balance_probs > 0.5).astype(int).flatten()


In [ ]:
# Compute confusion matrix and classification report
conf_matrix_hrv_balance = confusion_matrix(y_test_hrv_balance, y_pred_hrv_balance)
print("\nConfusion Matrix (HRV Balance):")
print(conf_matrix_hrv_balance)
class_report_hrv_balance = classification_report(y_test_hrv_balance, y_pred_hrv_balance)
print("\nClassification Report (HRV Balance):")
print(class_report_hrv_balance)


In [ ]:
# Visualize training and validation metrics
plt.figure(figsize=(12, 4))

# Plot training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='green', linestyle='--')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='green', linestyle='--')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
# ROC Curve and AUC

# Compute ROC curve and ROC area
fpr, tpr, thresholds = roc_curve(y_test_hrv_balance, y_pred_hrv_balance_probs)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


In [ ]:
# Precision, Recall, F1-Score Visualization

# Compute precision, recall, and F1-score
precision = precision_score(y_test_hrv_balance, y_pred_hrv_balance)
recall = recall_score(y_test_hrv_balance, y_pred_hrv_balance)
f1 = f1_score(y_test_hrv_balance, y_pred_hrv_balance)

# Bar chart for performance metrics
metrics = ['Precision', 'Recall', 'F1-Score']
values = [precision, recall, f1]

plt.figure(figsize=(6, 4))
sns.barplot(x=metrics, y=values, palette='Blues_d')
plt.title('Model Performance Metrics')
plt.ylim(0, 1)
for i, value in enumerate(values):
    plt.text(i, value + 0.02, f"{value:.2f}", ha='center')
plt.show()


<a name="interpretability"></a>
## 7. Model Interpretability

We will use SHAP values, LIME, and saliency maps to interpret the model's predictions and understand the important features contributing to the decision-making process.

In [ ]:
# SHAP Values Visualization

# Due to computational constraints, we'll use a small subset
X_test_sample = X_test_hrv_balance[:100]
y_test_sample = y_test_hrv_balance[:100]

# Create a DeepExplainer object for the trained model
explainer = shap.DeepExplainer(model, X_train_hrv_balance[:100])
shap_values = explainer.shap_values(X_test_sample)

# Visualize the SHAP values for a sample of test data
shap.image_plot(shap_values, -X_test_sample)


In [ ]:
# LIME Interpretation

from skimage.segmentation import mark_boundaries

# Convert grayscale images to RGB (duplicate channels) for LIME
X_test_rgb = np.repeat(X_test_hrv_balance[:10], 3, axis=-1)

# Initialize LIME image explainer
lime_explainer = lime_image.LimeImageExplainer()

# Function to make predictions for LIME
def predict_fn(images):
    images = images[..., 0].reshape(-1, 32, 32, 1)
    return model.predict(images)

# Explain a single image
explanation = lime_explainer.explain_instance(
    X_test_rgb[0].astype('double'),
    predict_fn,
    top_labels=1,
    hide_color=0,
    num_samples=1000
)

# Get image and mask from the explanation
temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0],
    positive_only=True,
    num_features=5,
    hide_rest=False
)

# Display the explanation
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.title('LIME Explanation')
plt.axis('off')
plt.show()


In [ ]:
# Saliency Maps using tf-keras-vis

# Define a model modifier to replace the last activation with linear
def model_modifier_function(cloned_model):
    cloned_model.layers[-1].activation = tf.keras.activations.linear

# Create a Saliency object
saliency = Saliency(model,
                    model_modifier=model_modifier_function,
                    clone=True)

# Define loss function for saliency
def loss_function(output):
    return output[:, 0]

# Generate saliency maps for a subset of test data
saliency_maps = saliency(loss_function, X_test_hrv_balance[:10])
saliency_maps = np.abs(saliency_maps)

# Plot original images and their corresponding saliency maps
plt.figure(figsize=(20, 4))
for i in range(10):
    # Original image
    plt.subplot(2, 10, i + 1)
    plt.imshow(X_test_hrv_balance[i].squeeze(), cmap='gray')
    plt.axis('off')
    # Saliency map
    plt.subplot(2, 10, i + 11)
    plt.imshow(saliency_maps[i].squeeze(), cmap='hot')
    plt.axis('off')
plt.suptitle('Original Images and Saliency Maps')
plt.show()


<a name="dimensionality"></a>
## 8. Alternative Modeling with Dimensionality Reduction

We will explore alternative modeling approaches by applying Principal Component Analysis (PCA) and t-Distributed Stochastic Neighbor Embedding (t-SNE) for dimensionality reduction and visualization.

In [ ]:
# Flatten the GAF images for PCA
X_flat = X_gaf_hrv_balance.reshape(X_gaf_hrv_balance.shape[0], -1)


In [ ]:
# Apply PCA to reduce dimensionality
n_components = 50
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_flat)


In [ ]:
# Visualize explained variance
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='b')
plt.title('Cumulative Explained Variance by PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid()
plt.show()


In [ ]:
# t-SNE Visualization
tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
X_tsne = tsne.fit_transform(X_pca)

# Plot t-SNE result
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=y_gaf_hrv_balance, palette='viridis')
plt.title('t-SNE Visualization of GAF Images')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Risk Label', loc='best')
plt.show()


In [ ]:
# Train a model on PCA-transformed data
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
    X_pca, y_gaf_hrv_balance, test_size=0.2, random_state=42, stratify=y_gaf_hrv_balance
)

# Define a simple neural network
model_pca = Sequential([
    Dense(64, activation='relu', input_shape=(n_components,)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model_pca.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history_pca = model_pca.fit(X_train_pca, y_train_pca, epochs=20, validation_data=(X_test_pca, y_test_pca), batch_size=32)


In [ ]:
# Evaluate the model
test_loss_pca, test_accuracy_pca = model_pca.evaluate(X_test_pca, y_test_pca)
print(f'PCA Model - Test Accuracy: {test_accuracy_pca}, Loss: {test_loss_pca}')


In [ ]:
# Generate predictions
y_pred_pca_probs = model_pca.predict(X_test_pca)
y_pred_pca = (y_pred_pca_probs > 0.5).astype(int).flatten()


In [ ]:
# Confusion matrix and classification report
conf_matrix_pca = confusion_matrix(y_test_pca, y_pred_pca)
print("\nConfusion Matrix (PCA Model):")
print(conf_matrix_pca)
class_report_pca = classification_report(y_test_pca, y_pred_pca)
print("\nClassification Report (PCA Model):")
print(class_report_pca)


In [ ]:
# Plot training history for PCA model
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history_pca.history['accuracy'], label='Training Accuracy', color='blue')
plt.plot(history_pca.history['val_accuracy'], label='Validation Accuracy', color='green', linestyle='--')
plt.title('Training vs Validation Accuracy (PCA Model)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history_pca.history['loss'], label='Training Loss', color='blue')
plt.plot(history_pca.history['val_loss'], label='Validation Loss', color='green', linestyle='--')
plt.title('Training vs Validation Loss (PCA Model)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()


<a name="comparison"></a>
## 9. Comparison of Models with Different PCA Components

We will compare the performance of models trained with different numbers of PCA components to understand the impact of dimensionality on model accuracy.

In [ ]:
# Function to train model with different number of PCA components
def train_model_with_pca_components(n_components_list, X_flat, y):
    results = {}
    for n_components in n_components_list:
        # Apply PCA
        pca = PCA(n_components=n_components)
        X_pca = pca.fit_transform(X_flat)
        # Split data
        X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
            X_pca, y, test_size=0.2, random_state=42, stratify=y
        )
        # Define model
        model_pca = Sequential([
            Dense(64, activation='relu', input_shape=(n_components,)),
            Dropout(0.5),
            Dense(32, activation='relu'),
            Dropout(0.5),
            Dense(1, activation='sigmoid')
        ])
        # Compile model
        model_pca.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        # Train model
        history_pca = model_pca.fit(X_train_pca, y_train_pca, epochs=20, validation_data=(X_test_pca, y_test_pca), batch_size=32, verbose=0)
        # Evaluate model
        test_loss_pca, test_accuracy_pca = model_pca.evaluate(X_test_pca, y_test_pca, verbose=0)
        results[n_components] = test_accuracy_pca
        print(f'Components: {n_components}, Test Accuracy: {test_accuracy_pca}')
    return results


In [ ]:
# Test different numbers of PCA components
n_components_list = [20, 30, 40, 50, 60]
results = train_model_with_pca_components(n_components_list, X_flat, y_gaf_hrv_balance)


In [ ]:
# Plot accuracy vs number of PCA components
plt.figure(figsize=(8, 6))
plt.plot(n_components_list, [results[n] for n in n_components_list], marker='o', linestyle='-', color='blue')
plt.title('Model Accuracy vs Number of PCA Components')
plt.xlabel('Number of PCA Components')
plt.ylabel('Test Accuracy')
plt.grid(True)
plt.show()


<a name="integrity"></a>
## 10. Data Integrity and Minimization

Data minimization is addressed by focusing on HRV metrics, which were identified as the most predictive and complete data available. This reduces the amount of data processed and stored, enhancing privacy and complying with data protection principles. Additionally, the use of sliding windows and GAF transformation condenses the time-series data into compact representations suitable for modeling, further contributing to data minimization.